In [1]:
import pandas as pd 
import numpy as np
import csv

from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.metrics import roc_curve, auc
from sklearn.utils import shuffle

import matplotlib.pyplot as plt
from tabulate import tabulate 

In [2]:
## load data sets
train_pd = pd.read_csv("~/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/data/Without Traffic Data/train_recid_M.csv")
test_pd = pd.read_csv("~/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward//data/Without Traffic Data/test_recid_M.csv")

## get rid of the record with 'p_age_first_offense' == 0
train_pd = train_pd.drop(['person_id', 'screening_date'], axis=1)
test_pd = test_pd.drop(['person_id', 'screening_date'], axis=1)
test_pd = test_pd[test_pd['p_age_first_offense'] != 0]

## split train and test
x_train, y_train = train_pd.values[:, :-1], train_pd.values[:, -1]
x_test, y_test = test_pd.values[:, :-1], test_pd.values[:, -1]

### CART

In [3]:
from sklearn.tree import DecisionTreeClassifier

### Cross Validation

In [4]:
def crossvalidation(X, Y, nfold, depth, min_sample_split=2, min_impurity_decrease=0, seed = 816):
    
    """
    
    @parameters:
    - X: training set -- features
    - Y: training set -- response variable
    - nfold: n-folds cross validation
    - depth: max split depth
    - min_sample_split
    - min_impurity_decrease
    - seed: random state
    
    """
    
    
    ## n-folds cross validation set up
    #cv = KFold(n_splits=nfold, random_state=seed, shuffle=True)
    cv = StratifiedKFold(n_splits=nfold, random_state=seed, shuffle=True)
    
    ## classifier: logistic regression
    classifier = DecisionTreeClassifier(max_depth=depth, min_samples_split= min_sample_split, 
                                        min_impurity_decrease=min_impurity_decrease, random_state=seed)
    train_acc, test_acc = [], []
    train_auc, test_auc = [], []

    i = 0
    for train, test in cv.split(X, Y):
    
        ## data & classifier
        X_train, Y_train = X[train], Y[train]
        X_test, Y_test = X[test], Y[test]
        fit_model = classifier.fit(X_train, Y_train)
    
        ## accuracy & probability
        train_acc.append(fit_model.score(X_train, Y_train))
        test_acc.append(fit_model.score(X_test, Y_test))
    
        train_prob = fit_model.predict_proba(X_train)[:,1]
        test_prob = fit_model.predict_proba(X_test)[:,1]
    
        ## compute AUC
    
        train_fpr, train_tpr, train_thresholds = roc_curve(Y_train, train_prob)
        test_fpr, test_tpr, test_thresholds = roc_curve(Y_test, test_prob)    
        train_auc.append(auc(train_fpr, train_tpr))
        test_auc.append(auc(test_fpr, test_tpr))
        i += 1

    return train_acc, test_acc, train_auc, test_auc

### Tune Parameters 
-- To prevent overfitting and get as good performance as possible.

-- criteria: difference between the avg. train accuracy and test accuracy and the difference between avg. train auc and avg. test auc are both smaller than 3%.

In [10]:
DEPTH = [1,2]
IMPURITY = [0, 0.001, 0.003, 0.005, 0.007, 0.009, 0.011]
SPLIT = [2,3,4,5,6,7,8,9] 

In [11]:
results = []

for d in DEPTH:
    for i in IMPURITY:
        for s in SPLIT:
            train_acc, test_acc, train_auc, test_auc = crossvalidation(x_train, y_train, 5, depth=d, min_impurity_decrease=i, min_sample_split=s)
            auc_diff = np.mean(train_auc) - np.mean(test_auc)
            results.append([d, i, s, np.mean(test_auc), auc_diff])

In [15]:
table = pd.DataFrame(results, columns=['Depth', 'Impurity Decrease','Min Split', 'Validation AUC', 'AUC Diff'])

In [16]:
table.sort_values(by = 'Validation AUC', axis=0, ascending =False)

Depth  Impurity Decrease  Min Split  Validation AUC  AUC Diff
56       2              0.000          2        0.584709  0.041382
71       2              0.001          9        0.584709  0.041382
58       2              0.000          4        0.584709  0.041382
59       2              0.000          5        0.584709  0.041382
60       2              0.000          6        0.584709  0.041382
61       2              0.000          7        0.584709  0.041382
62       2              0.000          8        0.584709  0.041382
63       2              0.000          9        0.584709  0.041382
64       2              0.001          2        0.584709  0.041382
65       2              0.001          3        0.584709  0.041382
66       2              0.001          4        0.584709  0.041382
67       2              0.001          5        0.584709  0.041382
68       2              0.001          6        0.584709  0.041382
69       2              0.001          7        0.584709  0.041382
70       2              0.001          8        0.584709  0.041382
57       2              0.000          3        0.584709  0.041382
77       2              0.003          7        0.584092  0.038448
79       2              0.003          9        0.584092  0.038448
78       2              0.003          8        0.584092  0.038448
76       2              0.003          6        0.584092  0.038448
75       2              0.003          5        0.584092  0.038448
74       2              0.003          4        0.584092  0.038448
73       2              0.003          3        0.584092  0.038448
72       2              0.003          2        0.584092  0.038448
80       2              0.005          2        0.578788  0.028402
87       2              0.005          9        0.578788  0.028402
86       2              0.005          8        0.578788  0.028402
85       2              0.005          7        0.578788  0.028402
81       2              0.005          3        0.578788  0.028402
83       2              0.005          5        0.578788  0.028402
..     ...                ...        ...             ...       ...
98       2              0.009          4        0.540792  0.023791
99       2              0.009          5        0.540792  0.023791
100      2              0.009          6        0.540792  0.023791
101      2              0.009          7        0.540792  0.023791
102      2              0.009          8        0.540792  0.023791
103      2              0.009          9        0.540792  0.023791
42       1              0.009          4        0.540792  0.023791
43       1              0.009          5        0.540792  0.023791
40       1              0.009          2        0.540792  0.023791
44       1              0.009          6        0.540792  0.023791
45       1              0.009          7        0.540792  0.023791
46       1              0.009          8        0.540792  0.023791
47       1              0.009          9        0.540792  0.023791
41       1              0.009          3        0.540792  0.023791
104      2              0.011          2        0.500000  0.000000
110      2              0.011          8        0.500000  0.000000
109      2              0.011          7        0.500000  0.000000
108      2              0.011          6        0.500000  0.000000
107      2              0.011          5        0.500000  0.000000
106      2              0.011          4        0.500000  0.000000
105      2              0.011          3        0.500000  0.000000
52       1              0.011          6        0.500000  0.000000
51       1              0.011          5        0.500000  0.000000
55       1              0.011          9        0.500000  0.000000
54       1              0.011          8        0.500000  0.000000
53       1              0.011          7        0.500000  0.000000
48       1              0.011          2        0.500000  0.000000
49       1              0.011          3        0.50

### Best Parameter
depth: 2 / Impurity Decrease: 0.005 / Min Split: 2

In [17]:
train_acc, test_acc, train_auc, test_auc = crossvalidation(x_train, y_train, 5, depth=2, min_impurity_decrease=0.005, min_sample_split=2)

In [18]:
np.mean(train_acc), np.mean(test_acc), np.mean(train_auc), np.mean(test_auc)

(0.7268075400752256, 0.726809314797451, 0.6071899229908797, 0.5787877945434585)

#### Heldout Test Set 
-- using 0.5 as threshold

In [19]:
cart = DecisionTreeClassifier(max_depth=2, min_samples_split=2, min_impurity_decrease=0.005, random_state=816).fit(x_train, y_train)
heldout_test_acc = cart.score(x_test, y_test)
heldout_test_acc

0.728494623655914

In [20]:
prob = cart.predict_proba(x_test)[:,1]
fpr,tpr,thresholds = roc_curve(y_test, prob)
heldout_test_auc = auc(fpr, tpr)
heldout_test_auc

0.565251543604545

-- using optimal threshold

#optimal_index = np.argmin(np.sqrt(np.square(1-tpr) + np.square(fpr)))
optimal_index = np.argmax(abs(tpr - fpr))
optimal_threshold = thresholds[optimal_index]
optimal_threshold

prediction = cart.predict_proba(x_test)[:,1]
heldout_test_acc = np.mean((prediction > optimal_threshold) == y_test)
heldout_test_acc

#### Log Results

In [21]:
#log model results to the model performance folder, as per standards
path = "C:\\Users\\binha\\Documents\\Duke\\Cynthia Research\\KY-analysis-mytrials\\broward\\broward models\\Baseline Model Results\\Without Traffic\\Misdemeanor\\"

train_auc_mean, train_auc_std = np.mean(train_auc), np.std(train_auc)
test_auc_mean, test_auc_std = np.mean(test_auc), np.std(test_auc)
                   
results = ["CART", train_auc_mean, train_auc_std, test_auc_mean, test_auc_std, heldout_test_auc, heldout_test_acc ]

with open(path + 'Misdemeanor Summary.csv', 'a') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerow(results)